<a href="https://colab.research.google.com/github/Kevinjchang98/lol-ml/blob/master/LoL_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q sklearn

In [ ]:
%tensorflow_version 2.x

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [ ]:
dfTrain = pd.read_csv('/content/train.csv')
dfEval = pd.read_csv('/content/eval.csv')

# Separate data used to classify from result we're looking for (survived or not)
yTrain = dfTrain.pop('win')
yEval = dfEval.pop('win')

In [ ]:
CATEGORICAL_COLUMNS = ['item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'perk0',	'perk0Var1',	'perk0Var2',	'perk0Var3',	'perk1',	'perk1Var1',	'perk1Var2',	'perk1Var3',	'perk2',	'perk2Var1',	'perk2Var2',	'perk2Var3',	'perk3',	'perk3Var1',	'perk3Var2',	'perk3Var3',	'perk4',	'perk4Var1',	'perk4Var2',	'perk4Var3',	'perk5',	'perk5Var1',	'perk5Var2',	'perk5Var3',	'perkPrimaryStyle',	'perkSubStyle',	'statPerk0',	'statPerk1',	'statPerk2']

# Create blank list
featureColumns = []

for featureName in CATEGORICAL_COLUMNS:
  vocabulary = dfTrain[featureName].unique() # Get unique values from given feature column as list
  featureColumns.append(tf.feature_column.categorical_column_with_vocabulary_list(featureName, vocabulary))


In [ ]:
# Input functions

def makeInputFn(dataDf, labelDf, numEpochs = 10, shuffle = True, batchSize = 32):
  def inputFunction() : #Inner function that is returned
    ds = tf.data.Dataset.from_tensor_slices((dict(dataDf), labelDf)) #Create tf.data.Dataset object that tensorflow needs
    if shuffle:
      ds = ds.shuffle(1000) # Shuffles order of data
    ds = ds.batch(batchSize).repeat(numEpochs) # Split dataset into batches and repeat by number of epochs we want
    return ds # Returns one batch of the ataset
  return inputFunction # Returns a function object to be used

trainInputFn = makeInputFn(dfTrain, yTrain)
evalInputFn = makeInputFn(dfEval, yEval, numEpochs=1, shuffle=False)

In [ ]:
linearEst = tf.estimator.LinearClassifier(feature_columns=featureColumns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp4rosazsr', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
# Building the model

linearEst.train(trainInputFn) # Trains

result = linearEst.evaluate(evalInputFn) # Get model metrics by testing on test data

clear_output()
print(result['accuracy'])
print(result)

1.0
{'accuracy': 1.0, 'accuracy_baseline': 0.50199205, 'auc': 1.0, 'auc_precision_recall': 1.0, 'average_loss': 0.008694667, 'label/mean': 0.50199205, 'loss': 0.008687625, 'precision': 1.0, 'prediction/mean': 0.5030141, 'recall': 1.0, 'global_step': 15710}


In [ ]:
# Predictions

result = list(linearEst.predict(evalInputFn))

print(dfEval.loc[20])
print(result[20]['probabilities'][1])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4rosazsr/model.ckpt-15710
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
item0               3857
item1               2055
item2               3050
item3               3067
item4                  0
item5               3009
item6               3364
perk0               8360
perk0Var1             10
perk0Var2              0
perk0Var3              0
perk1               8306
perk1Var1              3
perk1Var2              0
perk1Var3              0
perk2               8345
perk2Var1              3
perk2Var2              0
perk2Var3              0
perk3               8347
perk3Var1              0
perk3Var2              0
perk3Var3              0
perk4               8275
perk4Var1             16
perk4Var2              0
perk4Var3              0
perk5               8232
perk5Var1              0
perk5V